**Kelompok 9 - Butuh Libur**

6161901066 - Dennis Imanuel\
6161901104 - Putu Lila Kusumayani \
6161901114 - Bilqis Aulia R.\
6161901116 - Tiara Alamanda

In [ ]:
!pip install mysql-connector-python
import pandas as pd
import json
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 27.0 MB/s eta 0:00:00


In [ ]:
# Extract data from CSV file
def extract_csv(file_path):
    data = pd.read_csv(file_path)
    return data

In [ ]:
# Extract data from JSON file
def extract_json(file_path):
    with open(file_path, 'r') as file:
        json_data = json.load(file)
    data = pd.DataFrame(json_data)
    return data

In [ ]:
# Extract data from MySQL database
def extract_sql(query, connection_config):
    try:
        conn = mysql.connector.connect(**connection_config)
        data = pd.read_sql_query(query, conn)
    finally:
        if conn.is_connected():
            conn.close()
    return data

In [ ]:
# Transform the products data
def transform_products(data):
    data['product_name'] = data['product_name'].str.capitalize()
    data['category'] = data['category'].str.capitalize()
    return data

In [ ]:
# Transform the orders data
def transform_orders(data, products_data):
    data['quantity'] = data['quantity'].abs().astype(int)
    data = data.merge(products_data[['product_id', 'price']], on='product_id')
    data['total_value'] = data['quantity'] * data['price']
    data.drop(columns=['price'], inplace=True)
    return data

In [ ]:
# Transform the customers data
def transform_customers(data):
    data['email'] = data['email'].str.lower()
    return data

In [ ]:
# Transform and aggregate the orders data by customer
def transform_aggregate_orders(data):
    aggregated_data = data.groupby('customer_id').agg(
        total_orders=('order_id', 'count'),
        total_value=('total_value', 'sum')
    ).reset_index()
    aggregated_data['average_order_value'] = aggregated_data['total_value'] / aggregated_data['total_orders']
    return aggregated_data

In [ ]:
## Load the data into a MySQL database
def load(data, table_name, connection_config):
    try:
        engine = create_engine("mysql+mysqlconnector://{user}:{password}@{host}/{database}".format(**connection_config))
        data.to_sql(table_name, engine, if_exists='replace', index=False)
    finally:
        engine.dispose()

In [ ]:
# Execute ETL process
def main():
    connection_config = {
        'host': 'bgnwod8ayyih05dktlvh-mysql.services.clever-cloud.com',
        'user': 'ubluawk21geulf2z',
        'password': 'Y8hfbATYOoSQ3siMK5JX',
        'database': 'bgnwod8ayyih05dktlvh'
    }
    ##############################
    # Extract
    ##############################
    products_csv_data = extract_csv('products.csv')
    orders_json_data = extract_json('orders.json')
    customers_sql_data = extract_sql('SELECT * FROM customers', connection_config)

    ##############################
    # Transform
    ##############################
    products_csv_transformed = transform_products(products_csv_data)
    orders_json_transformed = transform_orders(orders_json_data, products_csv_transformed)
    customers_sql_transformed = transform_customers(customers_sql_data)

    customer_metrics = transform_aggregate_orders(orders_json_transformed)

    ##############################
    # Load
    ##############################
    load(products_csv_transformed, 'products', connection_config)
    load(orders_json_transformed, 'orders', connection_config)
    load(customers_sql_transformed, 'customers', connection_config)
    load(customer_metrics, 'customer_metrics', connection_config)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')